<a href="https://colab.research.google.com/github/superbunny38/2021DataScience/blob/main/7_%EA%B0%80%EC%84%A4%EA%B3%BC_%EC%B6%94%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/joelgrus/data-science-from-scratch.git book

Cloning into 'book'...
remote: Enumerating objects: 392, done.
remote: Total 392 (delta 0), reused 0 (delta 0), pack-reused 392
Receiving objects: 100% (392/392), 769.61 KiB | 4.69 MiB/s, done.
Resolving deltas: 100% (206/206), done.


In [2]:
%pwd

'/content'

In [3]:
!mkdir workspace

In [4]:
!mv /content/book/scratch workspace

In [5]:
%cd workspace

/content/workspace


### 예시: 동전 던지기

In [6]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float)->Tuple[float, float]:#재귀분포->이항분포
  mu = p*n#평균
  sigma = math.sqrt(p*(1-p)*n)#표준편차
  return mu, sigma

확률 변수가 정규분포를 따른다는 가정하에 normal_cdf를 사용하면 실제 동전던지기로부터 얻은 값이 구간 안(혹은 밖)에 존재할 확률을 계산할 수 있다.

In [9]:
from scratch.probability import normal_cdf

#누적 분포 함수는 확률 변수가 트정 값보다 작을 확률을 나타낸다
normal_probability_below = normal_cdf

In [10]:
def normal_probability_above(lo:float, mu:float = 0, sigma:float = 1)->float:
  return 1- normal_cdf(lo,mu,sigma)

In [11]:
def normal_probability_between(lo:float, hi:float, mu:float = 0,sigma:float=1)->float:
  return normal_cdf(hi,mu,sigma) - normal_cdf(lo, mu, sigma)

In [12]:
def normal_probability_outside(lo:float,
                               hi:float,
                               mu:float = 0,
                               sigma:float = 1)->float:
                               return 1 - normal_probability_between(lo,hi,mu,sigma)

In [13]:
from scratch.probability import inverse_normal_cdf
def normal_upper_bound(probability:float,
                       mu:float=0,
                       sigma:float = 1)->float:
                       return inverse_normal_cdf(probability, mu, sigma)#꼬리 부분

In [16]:
def normal_lower_bound(probability:float,
                            mu:float=0,
                            sigma:float=1)->float:
                            return inverse_normal_cdf(1-probability, mu, sigma)

In [17]:
def normal_two_sided_bounds(probability:float,
                            mu:float=0,
                            sigma:float=1)->float:
                            #입력한 probability 값을 포함하고 평균을 중심으로 대칭적인 구간을 반환
                            tail_probability = (1-probability)/2
                            upper_bound = normal_lower_bound(tail_probability, mu, sigma)
                            lower_bound = normal_upper_bound(tail_probability, mu, sigma)
                            return lower_bound, upper_bound

In [19]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000,0.5)#동전을 1000번 던져봄

In [21]:
mu_0#앞이 나올 평균(1000번 중 500번)

500.0

In [23]:
sigma_0#표준편차

15.811388300841896

In [24]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)#유의 수준: 5%
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


검정력(power)

In [26]:
#p가 0.5라고 가정할 때, 유의수준이 5%인 구간
lo, hi = normal_two_sided_bounds(0.95,mu_0,sigma_0)

In [29]:
print(lo,hi)

469.01026640487555 530.9897335951244


In [30]:
#p = 0.55인 경우(약간 편향)의 실제 평균과 표준편차
#앞면이 나올 확률이 55%일 때
mu_1, sigma_1 = normal_approximation_to_binomial(1000,0.55)#평균, 표준편차
print(mu_1, sigma_1)

550.0 15.732132722552274


In [31]:
#X가 주어진 구간 안에 존재할 경우
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1-type_2_probability
power

0.886548001295367

In [32]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi

526.0073585242053

In [34]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1-type_2_probability
power

0.9363794803307173

In [35]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    How likely are we to see a value at least as extreme as x (in either
    direction) if our values are from a N(mu, sigma)?
    """
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

two_sided_p_value(529.5, mu_0, sigma_0)   # 0.062

0.06207721579598835

In [36]:
import random

extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # Count # of heads
                    for _ in range(1000))                # in 1000 flips,
    if num_heads >= 530 or num_heads <= 470:             # and count how often
        extreme_value_count += 1                         # the # is 'extreme'


In [37]:
extreme_value_count

69

In [38]:

two_sided_p_value(531.5, mu_0, sigma_0)

0.046345287837786575

In [41]:
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below

In [42]:
upper_p_value(524.5, mu_0, sigma_0) # 0.061

0.06062885772582072

In [43]:
upper_p_value(526.5, mu_0, sigma_0) # 0.047

0.04686839508859242

신뢰구간 구하기

In [44]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)   # 0.0158


In [45]:
normal_two_sided_bounds(0.95, mu, sigma)        # [0.4940, 0.5560]

(0.4940490278129096, 0.5559509721870904)

In [46]:
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000) # 0.0158
normal_two_sided_bounds(0.95, mu, sigma) # [0.5091, 0.5709]

(0.5091095927295919, 0.5708904072704082)

p 해킹

In [47]:
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Using the 5% significance levels(신뢰구간)"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

In [48]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

In [49]:
num_rejections

46

### 예시: A/B test 해보기

In [50]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

In [51]:
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

1,000명 중 200명: 맛있어요!
1,000명 중 180명: 편향 없어요!

In [52]:
z = a_b_test_statistic(1000, 200, 1000, 180)    # -1.14
z

-1.1403464899034472

In [53]:
two_sided_p_value(z)#두 광고가 동일하게 효과적일 때 위 처럼(1000명 중 200, 1000명 중 180) 발생할 확률

0.2541419765422359

In [55]:
z = a_b_test_statistic(1000, 200, 1000, 150)    # -2.94
z

-2.948839123097944

In [57]:
two_sided_p_value(z) ##두 광고가 동일하게 효과적일 때 위 처럼(1000명 중 200, 1000명 중 150) 발생할 확률

0.003189699706216853

숫자 해설: 동일하기에 차이가 발생할 확률이 적음

## 베이즈 추론
: 만약 동전 앞면이 나타날 확률을 모른다면?

In [58]:
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

In [59]:
def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)